<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Mnist_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 727kB 17.2MB/s 
     |████████████████████████████████| 194kB 58.3MB/s 
     |████████████████████████████████| 1.2MB 27.0MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/4'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/2'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/0'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/8')]

In [5]:
folder = 'training/3'
(path/folder).ls()
(path/'testing').ls()

(#6131) [Path('/root/.fastai/data/mnist_png/training/3/46061.png'),Path('/root/.fastai/data/mnist_png/training/3/12329.png'),Path('/root/.fastai/data/mnist_png/training/3/37729.png'),Path('/root/.fastai/data/mnist_png/training/3/54603.png'),Path('/root/.fastai/data/mnist_png/training/3/18059.png'),Path('/root/.fastai/data/mnist_png/training/3/59158.png'),Path('/root/.fastai/data/mnist_png/training/3/10454.png'),Path('/root/.fastai/data/mnist_png/training/3/37786.png'),Path('/root/.fastai/data/mnist_png/training/3/59398.png'),Path('/root/.fastai/data/mnist_png/training/3/28174.png')...]

In [ ]:
# Loading the data and creating lists of tensors 
train_x = []
train_y = []
test_x = []
test_y = []
for digit in range(10):
    folder = 'training/' + str(digit)
    new_inp = [tensor(Image.open(x)) for x in (path/folder).ls()]
    #print(digit, len(new_inp))
    train_x += new_inp
    train_y += [digit]*len(new_inp)
    test_folder = 'testing/' + str(digit)
    test_inp = [tensor(Image.open(x)) for x in (path/test_folder).ls()]
    test_x += test_inp
    test_y += [digit]*len(test_inp)
    

In [8]:
# Creating the dataloaders and the batches for SGD
train_x_stacked = torch.stack(train_x).view(-1, 28*28).float()/255
train_y_stacked = tensor(train_y).view(-1, 1)
train_ds = list(zip(train_x_stacked, train_y_stacked))
train_dl = DataLoader(train_ds, batch_size= 256, shuffle=True)

test_x_stacked = torch.stack(test_x).view(-1, 28*28).float()/255
test_y_stacked = tensor(test_y).view(-1, 1)
test_ds = list(zip(test_x_stacked, test_y_stacked))
test_dl = DataLoader(test_ds, batch_size=256)

dls = DataLoaders(train_dl, test_dl)

train_x_stacked.shape, train_y_stacked.shape, test_x_stacked.shape, test_y_stacked.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [ ]:
df = pd.DataFrame(torch.trunc(tensor(train_x[2000]).float()))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

In [11]:
# Simple approach using cross entropy loss
def batch_accuracy(x, y):
    preds = torch.argmax(x, 1)
    correct = preds == y
    return correct.float().mean()

simple_net = nn.Sequential(
            nn.Linear(28*28, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 10)
)
loss = CrossEntropyLossFlat()
learn = Learner(dls, simple_net, opt_func = SGD, loss_func = loss, metrics = batch_accuracy)
learn.fit(10, 0.3)


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.355335,0.289755,0.838923,00:01
1,0.215078,0.283067,0.835513,00:01
2,0.172461,0.160481,0.872500,00:01
3,0.136123,0.139557,0.879755,00:01
4,0.117577,0.134357,0.879063,00:01
5,0.103849,0.119845,0.884728,00:01
6,0.094595,0.100268,0.890200,00:01
7,0.089639,0.114602,0.884616,00:01
8,0.078251,0.123225,0.884156,00:01
9,0.073833,0.105466,0.889218,00:01


In [34]:
# Manual implementation of the NN
def init_params(size1, size2): return torch.randn(size1, size2).requires_grad_()

def my_linear1(x): return x@w1 + b1

def my_linear2(x): return x@w2

def my_relu(x): return torch.maximum(tensor(0), x)
#def my_relu(x): return torch.where(x < 0, 0.01*x, x)

def my_softmax(x): 
    # Normalize the input to avoid numerical problems
    x = x - torch.max(x, dim = 1).values.view(-1, 1)
    return torch.exp(x) / torch.exp(x).sum(dim = 1).unsqueeze(1)

def my_loss(x, y): 
    x = my_softmax(x)
    y = y.view(-1, 1)
    selection = torch.gather(x, 1, y)
    #return (1-selection).mean()
    return (-torch.log(selection + 1e-15)).mean()

def my_model(x):
    lay1 = my_linear1(x)
    nonlin = my_relu(lay1)
    #return nonlin
    preds = my_linear2(nonlin)
    return(preds)

def update_grads(preds, y, params):
    loss = my_loss(preds, y)
    #print(loss)
    loss.backward()
    for p in params:
        my_grad = p.grad
        p.data -= my_grad*lr
        #print(p.mean())
        #print(my_grad.mean())
        p.grad.zero_()

def eval_batch():
    preds_test = my_model(test_x_stacked)
    positions = torch.max(preds_test, dim = 1).indices
    return (positions == test_y_stacked.view(1,-1)).float().mean()

w1 = init_params(28*28, 40)
#w1 = init_params(28*28, 10)
b1 = init_params(1, 40)
#b1 = init_params(1, 1)
w2 = init_params(40, 10)
#b2 = init_params(1, 1)
lr = 1
params = w1, b1, w2#, b2

In [35]:
for _ in range(40):
  for x,y in train_dl:
      preds = my_model(x)
      update_grads(preds, y, params)
  print(eval_batch())

tensor(0.7329)
tensor(0.8065)
tensor(0.8448)
tensor(0.8518)
tensor(0.8649)
tensor(0.9033)
tensor(0.8995)
tensor(0.8727)
tensor(0.9178)
tensor(0.9220)
tensor(0.9044)
tensor(0.9207)
tensor(0.9237)
tensor(0.9011)
tensor(0.9225)
tensor(0.9173)
tensor(0.9344)
tensor(0.9302)
tensor(0.9323)
tensor(0.9269)
tensor(0.9322)
tensor(0.9287)
tensor(0.9398)
tensor(0.9388)
tensor(0.9320)
tensor(0.9414)
tensor(0.9280)
tensor(0.9296)
tensor(0.9405)
tensor(0.9321)
tensor(0.9245)
tensor(0.9337)
tensor(0.9394)
tensor(0.9403)
tensor(0.9418)
tensor(0.9444)
tensor(0.9444)
tensor(0.9435)
tensor(0.9469)
tensor(0.9454)
